In [36]:
# TODO 
# 1. Add parsed fields to dictionary
# 2. wrap courses scraping into function returning dictionary
# 3. Append returned dictionaries to dictionary
# 4. create xmlifier function that accepts list
# 5. create jsonifier function that accepts list


def getCityCourseCount(course):
    # Find the number of City University of London courses shown on website
    # by scraping tag course-finder__results__summary
    from bs4 import BeautifulSoup
    import urllib.request as ur
    from xml.etree.ElementTree import Element, SubElement, Comment, tostring
    from xml.dom import minidom
    urlToScrape = "https://www.city.ac.uk/courses?level=" + course
    r = ur.urlopen(urlToScrape).read()
    soup = BeautifulSoup(r, "lxml")
    pageCount = soup.find_all('div', attrs={'class': 'course-finder__results__summary'})
    for courseListItem in pageCount:
        try:
            # we are expecting a span tag like this "<span>67 search results</span>"            
            myspan = courseListItem.find('span').text
            # we've now stripped the tags and have "67 search results"
            # we split the string into an array, using the default white space delimiter
            # and take the zeroeth element which is 67
            mycount = myspan.split()[0]
        except Exception as e:
            mycount = "0" # no results
    return mycount

citycourselist = ["Undergraduate", "Postgraduate", "Research Degrees"]
mycount = '67' # getCityCourseCount('Undergraduate')

i = 1
while i <= int(mycount):
    # we know that the general URL format will be
    # https://www.city.ac.uk/courses?level=Undergraduate&p=(n)
    # where paging is requested in querystring (text to the right of question mark)
    # attribute p, where (n) here represents the starting number
    # we know that paging increments in counts of 10 i.e. 1, 11, 21 and so forth
    # hence we count up to 61, the url then requests pages starting from 61, we know
    # that our count for undergraduate courses was 67 can we can stop there
    url = "https://www.city.ac.uk/courses?level=Undergraduate&p=" + str(i)
    r = ur.urlopen(url).read()
    soup = BeautifulSoup(r, "lxml")
    
    # get all the "div"s that ar of the class we are interested in.
    # note that the following line will return us a collection of "div" sections
    courseList = soup.find_all('div', attrs={'class': 'course-finder__results__item course-finder__results__item--undergraduate'})
    for courseListItem in courseList:
        
        # Here we cover everything with try/except constructs to ensure that we are not failing when an element is not there.    
        try:
            # first go inside the DIV to access the course name, note that it is under a <a> tag, so that's where we need to access here.
            courseNameElement = courseListItem.find('div', attrs={'class': "col-sm-24 col-md-18 col-lg-20"})
            courseName = courseNameElement.find('a').text
        except Exception as e:
            courseName = ""
                
        try:
            # here, we access the course description, you can find that it is under a DIV
            courseDescriptionElement = courseListItem.find('div', attrs={'class': "course-finder__results__item__description"})
            courseDescription = courseDescriptionElement.text
        except Exception as e:
            courseDescription = ""
        
        try:
            # now on to scraping the name of the school offering the course, this time under an <a> tag 
            courseSchoolElement = courseListItem.find('div', attrs={'class': "course-finder__results__item__md course-finder__results__item__md--school"})
            courseSchool = courseSchoolElement.find('a').text
        except Exception as e:
            courseSchool = ""

        try:
            # when we try to get the course code, we notice that there are two <span> sections under the section 
            # we are interested in, we want to get the second one, hence => .find_all('span')[1]

            courseCodeElement_1 = courseListItem.find('div', attrs={'class': "course-finder__results__item__md course-finder__results__item__md--code"})
            courseCodeElement_2 = courseCodeElement_1.find_all('span')[2]
            courseCode = courseCodeElement_2.text
        except Exception as e:
            courseCode = ""
        
        # And let's start populating the XML file by adding the scraped data one by one
        courseXML = SubElement(root, 'Course')
        
        courseNameXML = SubElement(courseXML, 'CourseName')
        courseNameXML.text = courseName
        
        courseDescriptionXML = SubElement(courseXML, 'CourseDescription')
        courseDescriptionXML.text = courseDescription
        
        courseSchoolXML = SubElement(courseXML, 'CourseSchool')
        courseSchoolXML.text = courseSchool
        
        courseSchoolXML = SubElement(courseXML, 'CourseCode')
        courseSchoolXML.text = courseCode
        
        # The following print lines are not needed since we will be saving to a file, 
        # but good to see the progress and for diagnostics
        print (courseName, i) 
        print (courseDescription)
        print (courseSchool)
        print (courseCode)
        print ("-------------")    
    # increment count
    i += 10

Accounting and Finance  1
Discover Cass Business School’s BSc (Hons) in Accounting and Finance. This degree is one of a small number of UK degrees supported by the Institute of Chartered Accountants in England and Wales. The programme introduces all areas of accounting and finance and can be a fantastic start to your career. This is extraordinary calling. Come and be a part of it.
Cass Business School
NN43 / N4N3 with placement
-------------
Actuarial Science  1
Discover Cass Business School’s BSc (Hons) in Actuarial Science. In this course you’ll delve into stochastic models, finance and financial reporting, business and, naturally, actuarial science itself. Find out more about the BSc (Hons) in Actuarial Science. This is extraordinary calling. Come and be a part of it.
Cass Business School
G322 / G321 with professional placement
-------------
Adult Nursing  1
The BSc (Hons) Adult Nursing degree course at City leads to eligibility to apply to the NMC and prepares you for a career in m

Investment and Financial Risk Management  41
Discover Cass Business School’s BSc (Hons) in Investment and Financial Risk Management. On this course, you’ll study every aspect of the discipline, from managing global investment portfolios to asset trading, as well as investment banking and many other areas of risk and investments. This is extraordinary calling. Come and be a part of it.
Cass Business School
N390 / N319 with professional placement
-------------
Journalism  41
Find out more about City's Journalism BA (Hons) degree, which provides highly relevant education and proficiency in print, broadcast and online journalism and relevant studies in humanities such as politics and the history of journalism.
School of Arts and Social Sciences
P500
-------------
Law  41
Our Law LLB degree provides you with the essential legal and academic skills to be successful in law as well as a range of transferable skills sought after in many other sectors.
The City Law School
M100
-------------
Lead

In [43]:
# Items in this list correspond to city url query string "level" e.g.
# https://www.city.ac.uk/courses?level=Executive Education
citycourselist = ["Undergraduate", "Postgraduate", "Research Degrees", "Foundation", "CPD", "Short Courses", "Executive Education"]
runningtotal = 0
for course in citycourselist:
    mycount = getCityCourseCount(course)
    print("Number of", course, "courses =", mycount)
    runningtotal += int(mycount)
print("Running total =", runningtotal)

Number of Undergraduate courses = 67
Number of Postgraduate courses = 142
Number of Research Degrees courses = 24
Number of Foundation courses = 9
Number of CPD courses = 251
Number of Short Courses courses = 133
Number of Executive Education courses = 15
Running total = 641
